In [1]:
import sys
import pathlib
path=pathlib.Path("../../../../robotics-course")
sys.path.append(str(path / 'build'))
import cv2 as cv
import numpy as np
import libry as ry
import time

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/04-opencv/solutions'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/04-opencv/solutions'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
#-- Add REAL WORLD configuration and camera
RealWorld = ry.Config()
RealWorld.addFile(str(path / "scenarios/pandasTable.g"))
ballRef = RealWorld.addFrame("ballRef")
ballRef.setPosition([0,0.4,.8])
ball = RealWorld.addFrame("ball",parent="ballRef",args="joint:transXY")
ball.setShape(ry.ST.sphere, [0.05])
ball.setColor([1,0,0])

In [3]:
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)

In [4]:
S.addSensor("camera")

In [5]:
def red_obj_det(image):
    result = image.copy()

    image_hsv = cv.cvtColor(image, cv.COLOR_RGB2HSV)

    # lower boundary RED color range values; Hue (0 - 10)
    lower1 = np.array([0, 100, 20])
    upper1 = np.array([10, 255, 255])

    # upper boundary RED color range values; Hue (160 - 180)
    lower2 = np.array([160,100,20])
    upper2 = np.array([179,255,255])

    lower_mask = cv.inRange(image_hsv, lower1, upper1)
    upper_mask = cv.inRange(image_hsv, lower2, upper2)

    full_mask = lower_mask + upper_mask;

    result = cv.bitwise_and(result, result, mask=full_mask)

    mask_greyscale = np.repeat(full_mask[:,:,np.newaxis],3,axis=2)
    
    contours, hierarchy = cv.findContours(cv.cvtColor(mask_greyscale, cv.COLOR_RGB2GRAY),cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
    return contours, full_mask



In [6]:
cameraFrame = RealWorld.frame("camera")
# camera = S.selectSensor("camera")
# camera
cameraFrame

In [7]:
# camera = S.selectSensor("camera")


In [27]:
cameraFrame = RealWorld.getFrame("camera")
# camera = S.selectSensor("camera")

f=0.895; W=640; H=360; #from the scenario file
                
fx, fy = f*H, f*H
px, py = W/2, H/2
R, off = cameraFrame.getRotationMatrix(), cameraFrame.getPosition()
R,off

(array([[ 1.       ,  0.       ,  0.       ],
        [ 0.       ,  0.8660254, -0.5      ],
        [ 0.       ,  0.5      ,  0.8660254]]),
 array([-0.01, -0.2 ,  1.8 ]))

In [28]:
pos = np.array([1,2,3])
m  = np.array(np.arange(9)).reshape(3,3)
print(m,pos)

[[0 1 2]
 [3 4 5]
 [6 7 8]] [1 2 3]


In [29]:
def pos_image_to_world(cameraFrame, x, y, Z):
## depth is sign-fliped, j: right, i: down

    ## Camera Frame
    pos = np.zeros(3)
    pos[0] = Z * (x - px) / fx;
    pos[1] = -Z * (y - py) / fy;
    pos[2] = -Z
    
    
    
    ## Coordinate transformation (from camera to world) 
    pos = (R@pos[:,np.newaxis]).flatten() + off
    
    return pos

In [30]:
def z_from_mask(depth, mask):
    return depth[mask!=0].mean()

In [31]:
def z_from_contours(depth, contours):
    c = contours[0].reshape(-1,2)
    return depth[c[:,1], c[:,0]].mean()

In [32]:
def estimateBallPosition(rgb, depth):
    contours, full_mask = red_obj_det(rgb)
#     print(contours)
#     Z_mean = z_from_mask(depth, full_mask)
    Z_mean = z_from_contours(depth, contours)
#     print("Pos Z: {}".format(Z_mean_b - Z_mean))
    
    M = cv.moments(contours[0])
#     print(M)
    if (M["m00"]):
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    
        pos = pos_image_to_world(cameraFrame, cX, cY, Z_mean)
    else:
        pos = np.zeros(3)
        print("Error: Ball not detected!")
    
    return pos 

In [33]:
# [rgb, depth] = S.getImageAndDepth()  #we don't need image
# pos = estimateBallPosition(rgb,depth)
# contours, full_mask = red_obj_det(rgb)
# c = contours[0].reshape(-1,2)
# depth[c[:,0], c[:,1]].mean()
# tmp = RealWorld.addFrame("pc")
# tmp.setShape(ry.ST.sphere, [0.01])
# tmp.setColor([255,0,5])
# #     tmp.setColor(rgb[i,j]/255)
# tmp.setPosition(pos.flatten())

In [34]:
tau = 1e-2
q = S.get_q()
w = .01
A = .3

tmp = RealWorld.addFrame("pc")
tmp.setShape(ry.ST.sphere, [0.05])
tmp.setColor([255,20,0,0.5])
for t in range(10000):

    #grab sensor readings from the simulation
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        
        est_pos = estimateBallPosition(rgb, depth)
        

    #     tmp.setColor(rgb[i,j]/255)
        tmp.setPosition(est_pos)
    
        gt_pos = ball.getPosition()
        err = np.linalg.norm(est_pos-gt_pos)        
        print("Error: ", err)
#         time.sleep(2)

    q[-2] = A * (np.cos(w*t) -1)
    q[-1] = A * np.sin(w*t)
    
    S.step(q, tau, ry.ControlMode.position)
    time.sleep(tau)

Error:  0.013424644706911968
Error:  0.013371803928828557
Error:  0.014001545970906247
Error:  0.01392924297355683
Error:  0.013220562574330783
Error:  0.013769455055694848
Error:  0.014328297128201927
Error:  0.013585930187853932
Error:  0.013931238333971974
Error:  0.01316331122235823
Error:  0.012749912245159424
Error:  0.01404153454491326
Error:  0.013623604834728305
Error:  0.01341884759380098
Error:  0.013671471449800222
Error:  0.012853963708951425
Error:  0.012008876251369236
Error:  0.013572665101416031
Error:  0.012213515914180928
Error:  0.012980407243226187
Error:  0.012990890146456949
Error:  0.01307093710058038
Error:  0.01239905495701654
Error:  0.012167467723546377
Error:  0.011571208300610066
Error:  0.012273132862743648
Error:  0.012852307188695728
Error:  0.01212841303392906
Error:  0.011772013727022773
Error:  0.011469870341322184
Error:  0.011873456287280296
Error:  0.010901746985518879
Error:  0.011383761362973773
Error:  0.011462849724980373
Error:  0.01057195661

Error:  0.010894416798059185
Error:  0.010688014608427866
Error:  0.010566650114410929
Error:  0.011689830588928465
Error:  0.011664672007153562
Error:  0.010340686676119724
Error:  0.011358154701650603
Error:  0.010107738592594957
Error:  0.01080722635654911
Error:  0.011013924957886792
Error:  0.010525449764701163
Error:  0.01145088807829625
Error:  0.011880289455023806
Error:  0.011634269250105288
Error:  0.012266776194731708
Error:  0.013074028314899663
Error:  0.012582938394259995
Error:  0.012429071272606954
Error:  0.012541716339187005
Error:  0.013077355989204236
Error:  0.012617011246186582
Error:  0.012802036495359282
Error:  0.01313961752609092
Error:  0.012423434183037974
Error:  0.012656137993436782
Error:  0.012380783449117428
Error:  0.012955800478855922
Error:  0.012770686379220365
Error:  0.012401491407757922
Error:  0.013635549848754437
Error:  0.013064655583333798
Error:  0.012861967854714398
Error:  0.01270062428427553
Error:  0.013186095912776308
Error:  0.01364661

Error:  0.013844709579191084
Error:  0.013732019575966915
Error:  0.013777938410767892
Error:  0.014689452705100582
Error:  0.013768846224469708
Error:  0.013341582758353001
Error:  0.013043875022891182
Error:  0.01308105138659865
Error:  0.01360297124808852
Error:  0.013296284447298001
Error:  0.013057686441218143
Error:  0.013305369989025234
Error:  0.012581074306354087
Error:  0.01334584314080389
Error:  0.01300884910963584
Error:  0.013146809641795777
Error:  0.012873056741924847
Error:  0.01315076359452472
Error:  0.011932995732116796
Error:  0.012091447720593748
Error:  0.012068271010282794
Error:  0.011863362116261553
Error:  0.012036449259347062
Error:  0.012194448176760053
Error:  0.01188528303718815
Error:  0.011423999666286272
Error:  0.012100992657782214
Error:  0.011053301444280203
Error:  0.011656287660284297
Error:  0.010336779069530455
Error:  0.011377815916811482
Error:  0.011089158553970335
Error:  0.010802513068426422
Error:  0.010618941097537095
Error:  0.0107139464

Error:  0.010614731212804497
Error:  0.009917564004529474
Error:  0.010992882092695378
Error:  0.010570001740717193
Error:  0.010858031065893956
Error:  0.011251522424543197
Error:  0.011696488623837506
Error:  0.010972367827779137
Error:  0.012805429851535061
Error:  0.011746344841493214
Error:  0.011487109636908072
Error:  0.012986021990457787
Error:  0.012995962573223694
Error:  0.013342141742694982
Error:  0.012379234434145691
Error:  0.013004416365739896
Error:  0.012970358082173483
Error:  0.012823914745827567
Error:  0.012694316807384615
Error:  0.013344845492043386
Error:  0.012707150541288494
Error:  0.011981878009132136
Error:  0.01330530354350489
Error:  0.01259150258683449
Error:  0.01301360049631453
Error:  0.012779659660290325
Error:  0.01316271869757072
Error:  0.012432827311782963
Error:  0.013236253478689602
Error:  0.013466105126050664
Error:  0.012675966824232218
Error:  0.014188190636078714
Error:  0.013995372996933321
Error:  0.013740871843255125
Error:  0.01396060